In [3]:
# model_comparison.ipynb or model_comparison.py

import pandas as pd

# Paths to leaderboard files (exported by arima_model.py, prophet_model.py, lstm_model.py)
path_arima = "C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/final project/app/data/results/arima_leaderboard.csv"
path_prophet = "C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/final project/app/data/results/prophet_leaderboard.csv"
path_lstm = "C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/final project/app/data/results/lstm_leaderboard.csv"

# Load leaderboards
leaderboard_df = pd.read_csv(path_arima)
leaderboard_prophet = pd.read_csv(path_prophet)
leaderboard_lstm = pd.read_csv(path_lstm)

# Combine with model name tagging
combined = pd.concat([
    leaderboard_df.assign(Model='ARIMA'),
    leaderboard_prophet.assign(Model='Prophet'),
    leaderboard_lstm.assign(Model='LSTM')
], ignore_index=True)

# Reorder columns correctly
final_leaderboard = combined[['Model', 'Scheme_Code', 'Fund_Name', 'MAE', 'RMSE', 'R2_Score']]

# Sort by Fund_Name then RMSE for cleaner comparison
final_leaderboard = final_leaderboard.sort_values(by=['Fund_Name', 'RMSE'])

# Display leaderboard
print("\nFinal Model Comparison Leaderboard:")
print(final_leaderboard.head())

# Save to CSV
output_path = "C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/final project/app/data/results/final_model_comparison.csv"
final_leaderboard.to_csv(output_path, index=False)
print(f"Final comparison saved to: {output_path}")

# Best model per fund
best_models = final_leaderboard.loc[final_leaderboard.groupby('Fund_Name')['RMSE'].idxmin()].reset_index(drop=True)
print("\nBest Forecasting Model Per Fund:")
print(best_models[['Fund_Name', 'Model', 'RMSE']].head())

# Generate recommendations
recommendations = []
for fund in final_leaderboard['Fund_Name'].unique():
    fund_data = final_leaderboard[final_leaderboard['Fund_Name'] == fund]
    best_model_row = fund_data.sort_values(by='RMSE').iloc[0]
    
    rec = {
        'Fund_Name': fund,
        'Recommended_Model': best_model_row['Model'],
        'Reason': f"{best_model_row['Model']} gave best RMSE ({best_model_row['RMSE']:.2f}) and R² ({best_model_row['R2_Score']:.2f})"
    }
    recommendations.append(rec)

rec_df = pd.DataFrame(recommendations)

# Save recommendations
rec_output = "C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/final project/app/data/results/model_recommendations.csv"
rec_df.to_csv(rec_output, index=False)
print(f"\n Recommendations saved to: {rec_output}")



Final Model Comparison Leaderboard:
     Model  Scheme_Code                                          Fund_Name  \
8     LSTM       100047         Aditya Birla Sun Life Liquid Fund - Growth   
2    ARIMA       100047         Aditya Birla Sun Life Liquid Fund - Growth   
5  Prophet       100047         Aditya Birla Sun Life Liquid Fund - Growth   
4  Prophet       100048  Aditya Birla Sun Life Liquid Fund -Institution...   
7     LSTM       100048  Aditya Birla Sun Life Liquid Fund -Institution...   

     MAE   RMSE  R2_Score  
8  1.040  1.310     0.978  
2  3.492  5.094     0.665  
5  4.911  6.517     0.454  
4  0.027  0.039     0.284  
7  0.030  0.040     0.248  
Final comparison saved to: C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/final project/app/data/results/final_model_comparison.csv

Best Forecasting Model Per Fund:
                                           Fund_Name    Model   RMSE
0         Aditya Birla Sun Life Liquid Fund - Growth     LSTM  1.310
1  Adity